In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#reading the file
df = pd.read_csv("train.csv")

In [ ]:
#top 5 rows
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
#shape of df
df.shape

(159571, 8)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#spliting the target variable
x = df['comment_text']
y = df[df.columns[2:3]].values

In [ ]:
#creating a function to clean the comments
def clean_text(text):
  word_arr=[]
  #converting sentance to words
  for comment in text:
    word_arr.append(word_tokenize(comment))

  punc_free=[]
  #removing punctuation marks
  for words in word_arr:
      w=[]
      for word in words:
        if word.isalpha():
          w.append(word)
      punc_free.append(w)

  stop_remove =[]
  #removing the stop words
  for words in punc_free:
      w=[]
      for word in words:
        if not word in stopwords.words('english'):
          w.append(word)
      stop_remove.append(w)

  final=[]
  #creating a string of all cleaned words
  for words in stop_remove:
      string=''
      for word in words:
        string=string+" "+word
      final.append(string[1:])

  return final


In [ ]:
#cleaing the data
x_clean=clean_text(x)

In [ ]:
#finding the maximun length of any comment
length=[]
for text in x_clean:
 length.append(len(text.split(" ")))

size=max(length)

In [ ]:
#encoding the text
vocab_size = size
encoded_text = [one_hot(d, vocab_size) for d in x_clean]

In [ ]:
#paddding the comments
max_length = size+50
padded_reviews = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [ ]:

df_test = pd.read_csv("test.csv")

In [ ]:
#cleaning the test data
clean_test=clean_text(df_test['comment_text'])

In [ ]:
length=[]
for text in clean_test:
 length.append(len(text.split(" ")))

size=max(length)

vocab_size = size
encoded_text = [one_hot(d, vocab_size) for d in clean_test]

max_len= size+50
test_reviews = pad_sequences(encoded_text, maxlen=max_len, padding='post')

In [ ]:
#creating a deep learing LSTM NN
def training(target):
  model = Sequential()
  model.add(Embedding(max_length, 32))
  model.add(Bidirectional(LSTM(32, activation='tanh')))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compiling the model and defining the early stoping
  model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
  #spliting the train test
  x_train, x_test, y_train, y_test = train_test_split(padded_reviews, df[target], test_size=0.3,random_state=42)
  history = model.fit(x_train, y_train, validation_split=0.2, epochs=1,batch_size=200,verbose=0,callbacks = es)
  accuracy=model.evaluate(x_test, y_test)
  #saving the model
  model.save(f"{target}.h5")

In [ ]:
for col in df.columns[2:]:
  training(col)

In [1]:
#getting the accuray , loss and prediction of test data
score=[]
loss_func=[]
predictions=[]
for col in df.columns[2:]:
  model = keras.models.load_model(f'{col}.h5')
  x_train, x_test, y_train, y_test = train_test_split(padded_reviews, df[col], test_size=0.3,random_state=42)
  mat=model.evaluate(x_test, y_test)
  score.append(mat[1])
  loss_func.append(mat[0])
  predictions.append(model.predict(test_reviews))

In [ ]:
#creating df for accuray and loss
df_ev = pd.DataFrame()
df_ev['columns']=df.columns[2:]
df_ev['accuracy']=score
df_ev['loss']=loss_func

In [ ]:
df_ev

,columns,accuracy,loss
0,toxic,0.888306,0.395242
1,severe_toxic,0.989848,0.076754
2,obscene,0.942555,0.257510
3,threat,0.997159,0.030021
4,insult,0.946879,0.235793
5,identity_hate,0.990976,0.059488


In [ ]:
#creating df for predictions
df_sub = pd.DataFrame()
df_sub['id']=df_test['id']
df_sub['toxic']=predictions[0]
df_sub['severe_toxic']=predictions[1]
df_sub['obscene']=predictions[2]
df_sub['threat']=predictions[3]
df_sub['insult']=predictions[4]
df_sub['identity_hate']=predictions[5]

In [ ]:
df_sub.head()

,id,toxic,severe_toxic,obscene,threat,insult
0,00001cee341fdb12,0.025844,0.000395,0.013902,0.000096,0.021413
1,0000247867823ef7,0.166335,0.003057,0.104591,0.000573,0.124947
2,00013b17ad220c46,0.052860,0.017070,0.042983,0.000231,0.017252
3,00017563c3f7919a,0.031536,0.000563,0.019510,0.000013,0.016121
4,00017695ad8997eb,0.087862,0.002021,0.035532,0.000062,0.028850


In [ ]:
#converting df into csv
df_sub.to_csv("submition_file.csv")